# 📘 analise_exploratoria
### **Objetivo:**
- Identificar padrões e relações entre presença (proxy por eventos), partidos e tempo; verificar coerências básicas entre fontes e levantar hipóteses para análises posteriores.

## Setup

In [8]:
# Pacotes
import json, os, re
from pathlib import Path
import numpy as np
import pandas as pd
import seaborn as sns

# Estilo básico (opcional)
sns.set_theme()
pd.set_option("display.max_columns", 100)

# Paths (ajuste se necessário)
ROOT = Path("..")  # este notebook está em lab/
DATA = ROOT / "data" / "processed"
LAB_DATA = ROOT / "lab" / "data" / "processed"

# Arquivos esperados
FILES = {
    "deputados": DATA / "deputados.json",
    "freq_eventos": DATA / "freq_eventos.csv",
    "eventos_sessoes":LAB_DATA / "eventos.csv",
    
    #"eventos_sessoes": DATA / "eventos_sessoes.csv",
    "votos_deputados": DATA / "votos_deputados.csv",
    "presencas": LAB_DATA / "presencas.csv",  # opcional
}


## Utilitários de leitura e tipagem

In [9]:
def exists(p: Path) -> bool:
    try:
        return p.exists() and p.is_file()
    except Exception:
        return False

def read_json_records(path: Path) -> pd.DataFrame | None:
    if not exists(path): 
        return None
    try:
        return pd.read_json(path, orient="records")
    except ValueError:
        with open(path, "r", encoding="utf-8") as f:
            data = json.load(f)
        return pd.json_normalize(data)

def read_csv(path: Path, **kwargs) -> pd.DataFrame | None:
    if not exists(path): 
        return None
    try:
        return pd.read_csv(path, **kwargs)
    except Exception as e:
        print(f"[WARN] Falha ao ler CSV {path}: {e}")
        return None

def coerce_int(series, allow_na=True):
    s = pd.to_numeric(series, errors="coerce")
    return s.astype("Int64") if allow_na else s.astype("int64", copy=False)


## Carregamento dos dados

In [10]:
df_deps   = read_json_records(FILES["deputados"])
df_freq   = read_csv(FILES["freq_eventos"])
df_evt    = read_csv(FILES["eventos_sessoes"])
df_votos  = read_csv(FILES["votos_deputados"])
df_pres   = read_csv(FILES["presencas"])  # pode ser None

# Tipagem mínima
if df_deps is not None and "id" in df_deps.columns:
    df_deps["id"] = coerce_int(df_deps["id"])

if df_freq is not None:
    if "id_deputado" in df_freq.columns:
        df_freq["id_deputado"] = coerce_int(df_freq["id_deputado"])
    if "num_eventos" in df_freq.columns:
        df_freq["num_eventos"] = pd.to_numeric(df_freq["num_eventos"], errors="coerce")

if df_evt is not None:
    # datas
    for c in ("dataHoraInicio", "dataHoraFim"):
        if c in df_evt.columns:
            df_evt[c] = pd.to_datetime(df_evt[c], errors="coerce")
    # normalizar id_evento se existir
    cand_evt = [c for c in df_evt.columns if c.lower() in {"id_evento","idevento","evento_id","id"}]
    if cand_evt:
        df_evt.rename(columns={cand_evt[0]:"id_evento"}, inplace=True)
        df_evt["id_evento"] = coerce_int(df_evt["id_evento"])

if df_pres is not None:
    # colunas padrão da rotina do 01
    for cc in ("id_evento","id_deputado"):
        if cc in df_pres.columns:
            df_pres[cc] = coerce_int(df_pres[cc])
    if "ano_origem" in df_pres.columns:
        df_pres["ano_origem"] = coerce_int(df_pres["ano_origem"])

# votos: detectar a coluna de id_deputado
v_id_dep_col = None
if df_votos is not None:
    cand = [c for c in df_votos.columns if ("deput" in c.lower() and "id" in c.lower()) or c.lower() in {"iddeputado","id_deputado","idparlamentar","idecadastro"}]
    if cand:
        v_id_dep_col = cand[0]
        df_votos[v_id_dep_col] = coerce_int(df_votos[v_id_dep_col])


## Views de análise

In [11]:
# Deputado + Proxy de presença (num_eventos)
view_dep_eventos = None
if df_deps is not None and df_freq is not None and "id" in df_deps.columns and "id_deputado" in df_freq.columns:
    view_dep_eventos = (
        df_freq.merge(
            df_deps[["id","nome","siglaPartido","siglaUf"]]
                 .rename(columns={"id":"id_deputado","siglaUf":"siglaUF"}),
            on="id_deputado", how="left"
        )
    )
view_dep_eventos.head(5) if view_dep_eventos is not None else "View indisponível"


,id_deputado,num_eventos,nome,siglaPartido,siglaUF
0,204379,908,Acácio Favacho,MDB,AP
1,220714,515,Adail Filho,REPUBLICANOS,AM
2,221328,441,Adilson Barroso,PL,SP
3,204560,722,Adolfo Viana,PSDB,BA
4,204528,1894,Adriana Ventura,NOVO,SP


## Cobertura temporal & volumetria

In [12]:
from datetime import datetime

periodo_evt = (None, None)
vol_evt_mensal = None
if df_evt is not None and "dataHoraInicio" in df_evt.columns:
    dt = df_evt["dataHoraInicio"].dropna()
    if len(dt):
        periodo_evt = (dt.min(), dt.max())
        vol_evt_mensal = df_evt.assign(mes=lambda x: x["dataHoraInicio"].dt.to_period("M")) \
                               .groupby("mes").size().reset_index(name="qtd_eventos")
        display(vol_evt_mensal.head(12))
periodo_evt


,mes,qtd_eventos
0,2020-02,74
1,2020-03,98
2,2020-04,33
3,2020-05,52
4,2020-06,50
5,2020-07,76
6,2020-08,66
7,2020-09,61
8,2020-10,40
9,2020-11,42


(Timestamp('2020-02-03 15:20:00'), Timestamp('2025-10-07 15:53:00'))

## Presença (proxy) — Top e Bottom por deputado

In [13]:
if view_dep_eventos is not None:
    top15 = view_dep_eventos.sort_values("num_eventos", ascending=False).head(15)
    bot15 = view_dep_eventos.sort_values("num_eventos", ascending=True).head(15)

    print("TOP 15 — por num_eventos")
    display(top15[["id_deputado","nome","siglaUF","siglaPartido","num_eventos"]])

    print("BOTTOM 15 — por num_eventos")
    display(bot15[["id_deputado","nome","siglaUF","siglaPartido","num_eventos"]])
else:
    print("View deputado-eventos indisponível.")


TOP 15 — por num_eventos


,id_deputado,nome,siglaUF,siglaPartido,num_eventos
74,178993,Carlos Henrique Gaguim,TO,UNIÃO,4957
231,204539,Hercílio Coelho Diniz,MG,MDB,2672
173,160575,Erika Kokay,DF,PT,2490
197,160598,Flávia Morais,GO,PDT,2355
138,178929,Diego Garcia,PR,REPUBLICANOS,2150
498,160518,Weliton Prado,MG,SOLIDARIEDADE,2126
177,178871,Evair Vieira de Melo,ES,PP,2072
155,204412,Dr. Zacharias Calil,GO,UNIÃO,2032
315,204455,Luiz Lima,RJ,NOVO,1965
4,204528,Adriana Ventura,SP,NOVO,1894


BOTTOM 15 — por num_eventos


,id_deputado,nome,siglaUF,siglaPartido,num_eventos
182,74065,Fatima Pelaes,AP,REPUBLICANOS,11
460,233598,Samuel Santos,GO,PODE,52
418,233594,Rafael Fera,RO,PODE,54
446,141533,Rodrigo Rollemberg,DF,PSB,79
398,233592,Paulo Lemos,AP,PSOL,86
382,230762,Pastor Claudio Mariano,PA,UNIÃO,90
170,230765,Enfermeira Rejane,RJ,PCdoB,153
288,231911,Lenir de Assis,PR,PT,160
504,230768,Zé Adriano,AC,PP,162
429,175765,Ribamar Silva,SP,PSD,164


## Comparativo por Partido (distribuição de num_eventos)

In [14]:
partido_stats = None
if view_dep_eventos is not None and "siglaPartido" in view_dep_eventos.columns:
    partido_stats = (
        view_dep_eventos.groupby("siglaPartido")["num_eventos"]
        .agg(qtd_deps="count", media="mean", mediana="median", p90=lambda s: s.quantile(0.90))
        .sort_values("media", ascending=False)
        .round(2)
        .reset_index()
    )
    display(partido_stats.head(12))
else:
    print("View deputado-eventos indisponível ou sem coluna 'siglaPartido'.")


,siglaPartido,qtd_deps,media,mediana,p90
0,REDE,1,1481.00,1481.0,1481.0
1,NOVO,5,1439.60,1661.0,1936.6
2,SOLIDARIEDADE,5,1287.20,1274.0,2024.4
3,PRD,5,1112.60,1215.0,1409.4
4,PDT,16,1080.44,927.0,1571.5
5,PCdoB,9,1033.89,1011.0,1532.8
6,PSOL,14,990.07,972.0,1667.5
7,PSDB,13,975.00,974.0,1440.0
8,PL,88,972.82,946.0,1481.3
9,PT,67,964.45,932.0,1431.0


## Comparativo por UF

In [15]:
uf_stats = None
if view_dep_eventos is not None and "siglaUF" in view_dep_eventos.columns:
    uf_stats = (
        view_dep_eventos.groupby("siglaUF")["num_eventos"]
        .agg(qtd_deps="count", media="mean", mediana="median", p90=lambda s: s.quantile(0.90))
        .sort_values("media", ascending=False)
        .round(2)
        .reset_index()
    )
    display(uf_stats.head(10))
else:
    print("View deputado-eventos indisponível ou sem coluna 'siglaUF'.")


,siglaUF,qtd_deps,media,mediana,p90
0,TO,8,1276.12,800.5,2192.0
1,MG,53,1035.79,911.0,1500.2
2,RS,31,1021.65,1049.0,1515.0
3,MS,8,1020.50,1013.5,1301.3
4,RN,8,976.88,1020.0,1255.1
5,ES,10,971.20,774.5,1535.6
6,DF,8,963.88,636.0,1869.1
7,BA,39,960.18,976.0,1400.0
8,AM,8,959.88,909.0,1693.8
9,GO,17,959.24,868.0,1592.2


## Votos vs. Presença (correlação simples)

In [16]:
corr_num_eventos_votos = np.nan
votos_por_dep = None

if df_votos is not None and v_id_dep_col is not None:
    votos_por_dep = (df_votos.groupby(v_id_dep_col).size().reset_index(name="qtd_votos")
                                 .rename(columns={v_id_dep_col: "id_deputado"}))
    display(votos_por_dep.head(10))

if (votos_por_dep is not None) and (view_dep_eventos is not None):
    join_corr = view_dep_eventos.merge(votos_por_dep, on="id_deputado", how="left").fillna({"qtd_votos":0})
    if join_corr["num_eventos"].notna().any() and join_corr["qtd_votos"].notna().any():
        corr_num_eventos_votos = join_corr[["num_eventos","qtd_votos"]].corr().iloc[0,1]
corr_num_eventos_votos


,id_deputado,qtd_votos
0,62881,577
1,66179,1006
2,66828,886
3,67138,600
4,68720,970
5,69871,916
6,72442,920
7,73433,939
8,73441,805
9,73460,824


np.float64(0.6615443722443114)

## Presenças (se disponível) — distribuição do tipo

In [17]:
pres_dist = None
if df_pres is not None and "tipo_presenca" in df_pres.columns:
    pres_dist = df_pres["tipo_presenca"].value_counts(dropna=False).to_frame("contagem")
    pres_dist["pct"] = (pres_dist["contagem"] / pres_dist["contagem"].sum() * 100).round(2)
    display(pres_dist.head(10))
else:
    print("presencas.csv não disponível ou sem coluna 'tipo_presenca'.")


,contagem,pct
tipo_presenca,,
NaN,564081,100.0


## Verificações rápidas de qualidade (duplicatas & integridade)

In [18]:
checks = []

# Duplicatas de IDs principais
if df_deps is not None and "id" in df_deps.columns:
    d = df_deps.duplicated(subset=["id"]).sum()
    checks.append(("deputados.id duplicado", int(d)))

if df_evt is not None and "id_evento" in df_evt.columns:
    d = df_evt.duplicated(subset=["id_evento"]).sum()
    checks.append(("eventos.id_evento duplicado", int(d)))

# Cobertura: freq_eventos vs deputados
cov_freq = np.nan
if (df_freq is not None) and (df_deps is not None) and "id" in df_deps.columns and "id_deputado" in df_freq.columns:
    deps_com_freq = df_freq["id_deputado"].nunique()
    deps_total    = df_deps["id"].nunique()
    cov_freq = round(deps_com_freq / deps_total * 100, 2) if deps_total else np.nan
    checks.append(("Cobertura freq_eventos (% de deputados)", cov_freq))

pd.DataFrame(checks, columns=["checagem","valor"])


,checagem,valor
0,deputados.id duplicado,0.0
1,eventos.id_evento duplicado,0.0
2,Cobertura freq_eventos (% de deputados),100.0


# 🧠 Hipóteses & próximos passos (texto)

- H1: Partidos/UFs com maior média de num_eventos também concentram maior qtd_votos por deputado.

- H2: Há sazonalidade na série mensal de eventos_sessoes.

- H3: A distribuição de tipo_presenca (quando disponível) se alinha ao volume de eventos mensais (consistência temporal).

## Próximos passos:

1. Normalizar datas e chaves onde necessário;

2. Explorar outliers (deputados com num_eventos muito baixo/alto);

3. Incorporar pesos por legislatura/mandato;

4. Cruzar remuneração/mandato com produtividade (métricas de “R$ por presença/voto”).

# Indicadores de EDA

In [20]:
# Defaults seguros
corr_num_eventos_votos = np.nan if "corr_num_eventos_votos" not in globals() else corr_num_eventos_votos
cov_freq = np.nan if "cov_freq" not in globals() else cov_freq

# Se ainda não tiver sido definido, deixe como (NaT, NaT)
if "periodo_evt" not in globals():
    periodo_evt = (pd.NaT, pd.NaT)

def fmt_periodo(pe):
    """Formata o período (min, max) de forma segura."""
    if isinstance(pe, tuple) and len(pe) == 2 and all((x is not None) and pd.notna(x) for x in pe):
        ini, fim = pe
        # aceita Timestamp ou datetime
        ini = pd.to_datetime(ini, errors="coerce")
        fim = pd.to_datetime(fim, errors="coerce")
        if pd.notna(ini) and pd.notna(fim):
            return f"{ini.date()} → {fim.date()}"
    return ""

periodo_txt = fmt_periodo(periodo_evt)

# Indicadores básicos
n_deps  = df_deps["id"].nunique() if (df_deps is not None and "id" in df_deps.columns) else 0
n_freq  = len(df_freq)   if df_freq  is not None else 0
n_evt   = len(df_evt)    if df_evt   is not None else 0
n_votos = len(df_votos)  if df_votos is not None else 0
n_pres  = len(df_pres)   if df_pres  is not None else 0

partidos_qtd = (
    df_deps["siglaPartido"].nunique()
    if (df_deps is not None and "siglaPartido" in df_deps.columns) else np.nan
)
ufs_qtd = (
    df_deps["siglaUf"].nunique()
    if (df_deps is not None and "siglaUf" in df_deps.columns) else np.nan
)

resumo_eda = pd.DataFrame([
    {"indicador": "Deputados (únicos)",                 "valor": n_deps},
    {"indicador": "freq_eventos (linhas)",              "valor": n_freq},
    {"indicador": "eventos_sessoes (linhas)",           "valor": n_evt},
    {"indicador": "votos_deputados (linhas)",           "valor": n_votos},
    {"indicador": "presencas (linhas)",                 "valor": n_pres},
    {"indicador": "Cobertura freq_eventos (% de deps)", "valor": cov_freq},
    {"indicador": "Período eventos",                    "valor": periodo_txt},
    {"indicador": "Partidos distintos",                 "valor": partidos_qtd},
    {"indicador": "UFs distintas",                      "valor": ufs_qtd},
    {"indicador": "Correlação num_eventos ~ qtd_votos", "valor": np.round(corr_num_eventos_votos, 3)},
])

# (opcional) formatação leve
num_cols = ["valor"]
for c in num_cols:
    if c in resumo_eda.columns:
        resumo_eda[c] = resumo_eda[c].apply(lambda x: x if isinstance(x, (int, float, np.number)) else x)

resumo_eda


,indicador,valor
0,Deputados (únicos),513
1,freq_eventos (linhas),513
2,eventos_sessoes (linhas),5340
3,votos_deputados (linhas),438290
4,presencas (linhas),564081
5,Cobertura freq_eventos (% de deps),100.0
6,Período eventos,2020-02-03 → 2025-10-07
7,Partidos distintos,20
8,UFs distintas,27
9,Correlação num_eventos ~ qtd_votos,0.662
